In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os

sub_types = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']


base_dir = '/home/wkeeley/Desktop/Brain-Hemorrhaging'
image_dir = os.path.join(base_dir, 'Images')
train_dir = os.path.join(image_dir, 'Train_Images')
val_dir = os.path.join(image_dir, 'Validation_Images')

dcm_dir = os.path.join(base_dir, 'stage_1_train_images')
dcm_test_dir = os.path.join(base_dir, 'stage_1_test_images')

sub_train_dirs = []
sub_val_dirs = []

for i in sub_types:
    sub_train_dirs.append(os.path.join(train_dir, i))
    sub_val_dirs.append(os.path.join(val_dir, i))



In [2]:
#Setup file hierarchy
for i in [base_dir, image_dir, train_dir, val_dir]:
    try:
        os.mkdir(i)
    except FileExistsError:
        pass

for i in sub_train_dirs:
    try:
        os.mkdir(i)
        os.mkdir(os.path.join(i, 'positive'))
        os.mkdir(os.path.join(i, 'negative'))
    except FileExistsError:
        pass

for i in sub_val_dirs:
    try:
        os.mkdir(i)
    except FileExistsError:
        pass
    

In [3]:
data = pd.read_csv('stage_1_train.csv')

In [35]:
ids = []

for i in range(0, len(data), 6):
    img_id = data.iloc[i, 0][:12]
    ids.append(img_id)


In [36]:
ids

['ID_63eb1e259',
 'ID_2669954a7',
 'ID_52c9913b1',
 'ID_4e6ff6126',
 'ID_7858edd88',
 'ID_205506026',
 'ID_293f0cec8',
 'ID_12cb04d93',
 'ID_9aef16bb2',
 'ID_2802a8ae2',
 'ID_db5acfff8',
 'ID_ca62b5929',
 'ID_4419f8ae9',
 'ID_bcf1add28',
 'ID_aeda0804d',
 'ID_86efc4c17',
 'ID_877720a2f',
 'ID_ec7b5c7c9',
 'ID_8e2968e3f',
 'ID_fde0f077f',
 'ID_b2ba0706d',
 'ID_12fb979b2',
 'ID_4d4fd56d8',
 'ID_bf0c67657',
 'ID_066f36ad6',
 'ID_1024b5019',
 'ID_3fb8e224c',
 'ID_1dd5ecddd',
 'ID_d94d066a0',
 'ID_f8fd7b533',
 'ID_0706bacd4',
 'ID_2739292a5',
 'ID_f42d20382',
 'ID_02e0adea2',
 'ID_25321a1f8',
 'ID_94ee87654',
 'ID_5fc337950',
 'ID_0c0b804cb',
 'ID_85c8dead7',
 'ID_e2602081e',
 'ID_df33594dc',
 'ID_56e893e7c',
 'ID_975b56ce2',
 'ID_8eda1b9c2',
 'ID_4f4c47ae0',
 'ID_ea2a04ec1',
 'ID_646e3e896',
 'ID_29785bd15',
 'ID_f7356ca2b',
 'ID_e70495f58',
 'ID_34dd43f1a',
 'ID_e21609940',
 'ID_080c7a005',
 'ID_e7ac4ea57',
 'ID_ba18c312c',
 'ID_d42c02f69',
 'ID_d5667854d',
 'ID_87d2a77f1',
 'ID_ff1730610

In [8]:
from zipfile import ZipFile
num_images = 674262 #From data viz

rsna = ZipFile('rsna-intracranial-hemorrhage-detection.zip')
rsna.getinfo('stage_1_train_images')

In [30]:
from windowing import window_image, get_windowing
import cv2


f = rsna.extract('stage_1_train_images/ID_94c3f5a91.dcm')
img_data = pydicom.read_file(f)
img = img_data.pixel_array
window_center, window_width, intercept, slope = get_windowing(img_data)
cv2.imwrite('test.jpg', window_image(img, 40, 80, intercept, slope))

True

In [ ]:
from windowing import window_image, get_windowing


for g in range(int(num_images*0.9)):
    i = ids[g]

    f = os.path.join('stage_1_train_images', i + '.dcm')
    file = rsna.extract(f)
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    
    for j in sub_types:
        if j == sub_types[0]:
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'negative'), i + '.jpg')
            
            cv2.imwrite(file_path, img)
            print(file_path)
        elif j == sub_types[1]:
            img = window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[1], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[1], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)   
        elif j == sub_types[2]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[2], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[2], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)
        elif j == sub_types[3]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[3], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[3], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)
        elif j == sub_types[4]:
            img = window_image(image, 80, 200, intercept, slope) #subdural window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[4], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[4], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)
    os.remove(file)

for g in range(int(0.9*num_images), num_images):
    i = ids[g]
    f = os.path.join('stage_1_train_images', i + '.dcm')
    file = rsna.extract(f)
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing_data(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    
    for j in sub_types:
        if j == sub_types[0]:
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'negative'), i + '.jpg')
            
            cv2.imwrite(file_path, img)

        elif j == sub_types[1]:
            img = window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[1], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[1], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)   
        elif j == sub_types[2]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[2], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[2], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)
        elif j == sub_types[3]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[3], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[3], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)
        elif j == sub_types[4]:
            img = window_image(image, 80, 200, intercept, slope) #subdural window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[4], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[4], 'negative'), i + '.jpg')
            cv2.imwrite(file_path, img)
    os.remove(file)

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_63eb1e259.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_2669954a7.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_52c9913b1.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_4e6ff6126.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_7858edd88.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_205506026.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_293f0cec8.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_12cb04d93.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_9aef16bb2.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_2802a8ae2.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/positive/ID_3a2a4cc6d.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/positive/ID_7e9c4d976.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_41ff04c56.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_270b42a2f.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_52051f1fe.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_0f8b7e93b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_26969210b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_87f1a0e49.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_0b3d472d9.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_449ea7e44.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_236ec63c3.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_65bdf3f8e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_9d9cc6b01.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_40b5fbfcf.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_143f53f8b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_4f3bd0fb6.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_6ef1c9a94.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_8719b5f00.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_4cef7c01e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_5e1d42bde.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_890145a5a.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_2b50f35d3.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_f0170417e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_8c5cda670.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_335550a51.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_a58b6319d.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_4d31bf72b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_b6e6c1979.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_8aa0b977e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_deb35a49b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_774221c8b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_bb4fa379d.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_d28fd120b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_3100da4af.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_05417db0f.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_11974335b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_60c7d1992.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_5c75a5a88.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_3eb0adf27.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_56bb8adfa.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

...

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_fdeee658e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_e8d32382b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_d825fe962.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_5dc4d7fc4.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_f94d98373.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_eeb334fdf.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_f94a1552a.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_ce4e11058.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_25705b682.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_504c9f607.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_4abae578e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_d90d114c2.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_f3b9c9251.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_10fe8fec6.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_8c1b5342a.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_ddabb600a.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_64cf46ee3.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_8be2985f0.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_c8ef17a86.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_1d3e05d0b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_d9ec6f7de.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_bc99663df.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_3eaacaed9.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_e44d5383e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_5f78f972b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_8b000c417.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_b7c2f8640.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_7555f187b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_74c23785e.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_3568a4a8b.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_0ba1fb7f2.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_65b896304.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_70322ce1d.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_6a94b73c9.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_728bd3407.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_13476438f.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_ae24ec5ea.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_99ed5d927.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_de4a06e27.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_071c749b9.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging

/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_65390609c.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_e36781f98.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_79107d7d3.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_c01252aff.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_9c2679d2f.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_499582ad2.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_1e66f1911.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_344c2e801.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_05792abe2.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Train_Images/epidural/negative/ID_2e8e2cde8.jpg
/home/wkeeley/Desktop/Brain-Hemorrhaging